# Init

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

import pyspark.sql.functions as func

# Dataset Imports

In [4]:
from datetime import datetime
start = datetime.now()

df_mentions = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .option("inferSchema", "true") \
    .load(path=["s3://gdelt-open-data/v2/mentions/*.mentions.csv"]) \
    .select(["_c1","_c4","_c5","_c13"]).distinct()
df_mentions = df_mentions.toDF(*["EventDate","MentionSource","MentionIdentifier","MentionDocTone"])
df_mentions.cache()

print(df_mentions.count())
print(df_mentions.printSchema())
print(datetime.now()-start)

329962831
root
 |-- EventDate: string (nullable = true)
 |-- MentionSource: string (nullable = true)
 |-- MentionIdentifier: string (nullable = true)
 |-- MentionDocTone: string (nullable = true)

None
0:20:44.537536


# Write Data to Parquet Format

In [5]:
df_mentions.write.parquet("s3://labadie-gdelt-tradewar/mentions.parquet", mode="overwrite")